In [3]:
def get_bitstream_segment(stream, start, end):
    return stream[start:end+1]

def find_template_match(stream, template, search_range):
    """
    Returns the index in `stream` where `template` is found.
    Limits search to given range (tuple of indices).
    Returns -1 if not found.
    """
    start, end = search_range
    for i in range(start, end - len(template) + 2):
        if stream[i:i+len(template)] == template:
            return i
    return -1

In [4]:


def recover_templates(B1, B2, TMS1, TMS2, l_min=3, l_max=9):
    """
    Recovers T1 and T2 templates from B1, B2, and known TMS1, TMS2.
    Each input is a binary string.
    """
    # Split TMS values into upper and lower 8-bit parts
    TMS1_upper, TMS1_lower = TMS1[:8], TMS1[8:]
    TMS2_upper, TMS2_lower = TMS2[:8], TMS2[8:]

    templates = [None, None]  # T1, T2
    starts = [0, 128]         # start indices for T1 and T2 recovery
    TMS_refs = [(TMS1_lower, TMS2_lower), (TMS1_upper, TMS2_upper)]

    for i in [0, 1]:  # 0 for T1, 1 for T2
        start = starts[i]
        TMS1_ref, TMS2_ref = TMS_refs[i]
        for l in range(l_min, l_max + 1):
            for num in range(2**l):
                candidate = format(num, f'0{l}b')
                p1 = find_template_match(B1, candidate, (start, start+127))
                p2 = find_template_match(B2, candidate, (start, start+127))
                if p1 != -1 and p2 != -1:
                    # Extract 8 bits after matched template
                    k1 = B1[p1 + l : p1 + l + 8]
                    k2 = B2[p2 + l : p2 + l + 8]
                    if k1 == TMS1_ref and k2 == TMS2_ref:
                        templates[i] = candidate
                        break
            if templates[i]:
                break

    if all(templates):
        return templates[0], templates[1]
    else:
        return None, None




In [5]:
# Example usage
import time

if __name__ == "__main__":
    # Start the timer
    start_time = time.time()

    # Example inputs (binary strings of length 256 for B1, B2; 16 for TMS)

    B1 = '0000101111001100100101001001011011110001111011001110100100010011001011100110111111111001110000010010000000101101000111111111011000100100100000001100101001010111011110100100011100011110100110101100111100101110001111101001110000000001100000011111011010100000'  # Provide a 256-bit binary string
    B2 = '0000100001011100100101001001011011110001111011001110100100010011001011100110111111111001110000010010000000101101000111111111011000100100100000001100001111000111011110100100011100011110100110101100111100101110001111101001110000000001100000011111011010100000'  # Provide a 256-bit binary string
    TMS1 = '1010010111110011'  # e.g., 0xA5F3
    TMS2 = '0011110000010111'  # e.g., 0x3C17

    T1, T2 = recover_templates(B1, B2, TMS1, TMS2)
    if T1 and T2:
        print(f"Recovered templates: T1 = {T1}, T2 = {T2}")
    else:
        print("Template recovery failed.")

    # End the timer
    end_time = time.time()
    # Calculate and print the elapsed time in milliseconds
    execution_time_ms = (end_time - start_time) * 1000
    print(f"Execution Time: {execution_time_ms:.3f} ms")

Recovered templates: T1 = 010, T2 = 1100
Execution Time: 0.200 ms
